 가상환경 삭제 : conda remove -n 가상환경이름 --all

In [2]:
pip install transformers

  Using cached transformers-4.20.1-py3-none-any.whl (4.4 MB)
     ---------------------------------------- 78.4/78.4 kB 4.3 MB/s eta 0:00:00
  Using cached huggingface_hub-0.8.1-py3-none-any.whl (101 kB)
     -------------------------------------- 151.6/151.6 kB 3.0 MB/s eta 0:00:00
  Using cached tokenizers-0.12.1-cp39-cp39-win_amd64.whl (3.3 MB)
     -------------------------------------- 262.8/262.8 kB 1.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import transformers
transformers.__version__

'4.20.1'

In [27]:
import pandas as pd 
import numpy as np
import torch # pytorch install 해야함
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import AutoTokenizer, TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [3]:
train = pd.read_csv('dataset/train.csv')

In [4]:
train_data, val_data = train_test_split(train, test_size=0.25, 
                                        stratify = train.target,
                                        random_state =0) # 25프로로 설정
# 전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화
train_data = train_data.reset_index().drop('index', axis=1)
val_data = val_data.reset_index().drop('index', axis=1)


In [5]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2',bos_token='</s>',eos_token='</s>',pad_token='<pad>')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
tokenizer.encode('안녕하세요. 저는 데이터분석가가 되고싶은 최선우 입니다')

[25906,
 8702,
 7801,
 25856,
 9265,
 7162,
 16071,
 15460,
 11510,
 10973,
 7898,
 8135,
 9212,
 7791,
 8092,
 9241,
 7172,
 7182]

In [7]:
tokenizer.tokenize('안녕하세요. 저는 데이터분석가가 되고싶은 최선우 입니다')

['▁안녕',
 '하',
 '세',
 '요.',
 '▁저',
 '는',
 '▁데이터',
 '분석',
 '가가',
 '▁되고',
 '싶',
 '은',
 '▁최',
 '선',
 '우',
 '▁입',
 '니',
 '다']

In [11]:
tokenizer.decode(tokenizer.encode('안녕하세요. 저는 데이터분석가가 되고싶은 최선우 입니다'))

'안녕하세요. 저는 데이터분석가가 되고싶은 최선우 입니다'

In [12]:
max_seq_len = 140 # 제일 긴 문장의 길이

In [13]:
tokenizer.encode('안녕하세요. 저는 데이터분석가가 되고싶은 최선우 입니다',max_length=max_seq_len,pad_to_max_length = True)

[25906,
 8702,
 7801,
 25856,
 9265,
 7162,
 16071,
 15460,
 11510,
 10973,
 7898,
 8135,
 9212,
 7791,
 8092,
 9241,
 7172,
 7182,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3]

In [14]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, data_labels = [], []
    
    for example, label in tqdm(zip(examples, labels), total=len(examples)):

        bos_token = [tokenizer.bos_token]
        eos_token = [tokenizer.eos_token]
        tokens = bos_token + tokenizer.tokenize(example) + eos_token
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        input_ids.append(input_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return input_ids, data_labels

In [15]:
train_data
val_data

,id,reviews,target
0,10749,심플하면서 깔끔하고 가격 저렴하고 배송 빠르고~ 최고네요 ^^,5
1,12258,재구매 아주 좋아요 손톱관리하기가 너무 좋아요~~,5
2,16351,조립도 쉽고 디자인 너무 예뻐요ㅎㅎ,5
3,9402,재구매 배송 빠르고 불량품 없어서 좋아요,5
4,7722,배송은 늦었지만 수령후 설치해서사용중인데 모니터 고정부분이 움직일때 좌우로 움직이네...,4
...,...,...,...
6245,24390,싸서샀는데별로임 질떨어짐,2
6246,16848,잘 받아 보았습니다,2
6247,20355,배송이 너무 늦게오네요..,2
6248,15497,향이...별로예요 ㅠㅠㅠ,1


In [16]:
train_X, train_y = convert_examples_to_features(train_data['reviews'], train_data['target'], max_seq_len=max_seq_len, tokenizer=tokenizer)


100%|██████████████████████████████████████████████████████████████████████████| 18750/18750 [00:02<00:00, 7699.38it/s]


In [17]:
val_X, val_y = convert_examples_to_features(val_data['reviews'], val_data['target'], max_seq_len=max_seq_len, tokenizer=tokenizer)


100%|████████████████████████████████████████████████████████████████████████████| 6250/6250 [00:00<00:00, 7705.41it/s]


In [18]:
tokenizer.decode(train_X[0])

'</s> 상품이 저가 원하는대로 왔어요.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [19]:
model = TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['lm_head.weight', 'transformer.h.2.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.5.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [21]:
layers = tf.keras.layers.Input(shape=(max_seq_len),dtype=tf.int32)
outputs = model([layers])

In [33]:
class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.classifier = tf.keras.layers.Dense(4,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='softmax',
                                                name='classifier')

    def call(self, inputs):
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs[0][:, -1]
        cls_token = self.dropout(cls_token)
        prediction = self.classifier(cls_token)

        return prediction

In [37]:
import tensorflow as tf
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

KeyError: 'COLAB_TPU_ADDR'